# Setup AML
In this notebook, we'll setup our Azure Machine Learning workspace as well as another storage account.
This notebook assumes you have az cli installed. It can be installed from [here](https://docs.microsoft.com/en-us/cli/azure/install-azure-cli?view=azure-cli-latest). If you already have setup a Workspace and storage account from the [portal](portal.azure.com), you can skip this notebook.

In [1]:
import azureml.core
from azureml.core import Workspace
from pathlib import Path
import json
import os

In [2]:
print("AML SDK Version:", azureml.core.VERSION)

AML SDK Version: 1.0.43


### Set up your Azure Machine Learning workspace

To create or access an Azure ML Workspace, you will need the following information:
 - Your subscription id
 - A resource group name
 - A name for your workspace
 - A region for your workspace
 
**Note**: As with other Azure services, there are limits on certain resources like cluster size associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

If you have a workspace created already, you need to get your subscription and workspace information. You can find the values for those by visiting your workspace in the [Azure portal](http://portal.azure.com). If you don't have a workspace, the create workspace command in the next section will create a resource group and a workspace using the names you provide.

In [6]:
# enter your info to create the workspace.

subscription_id = "---------------"
resource_group = "---------------"
workspace_name = "---------------"
workspace_region = "---------------"
account_name = "---------------"

### Create the workspace
This cell will create an AML workspace for you in a subscription, provided you have the correct permissions.

This will fail when:

1. You do not have permission to create a workspace in the resource group
2. You do not have permission to create a resource group if it's non-existing.
3. You are not a subscription owner or contributor and no Azure ML workspaces have ever been created in this subscription


If workspace creation fails, please work with your IT admin to provide you with the appropriate permissions or to provision the required resources. If this cell succeeds, you're done configuring AML!

In [ ]:
ws = Workspace.create(
    name=workspace_name,
    subscription_id=subscription_id,
    resource_group=resource_group,
    location=workspace_region,
    create_resource_group=False,
    exist_ok=True
)

In [ ]:
ws.get_details()

ws.write_config()
import shutil

shutil.copy('config.json','../notshared/')

### Setup a Storage Account

One of the resources that will have been generated when created the workspace is a storage account. This storage account is used as the default datastore for Azure Machine Learning. However, to keep things seperate, we'll create another storage account for holding input and output data. This will also let us walk through how to connect our workspace to other storage locations.

In [ ]:
container_name = "aml"
file_share_name = "chestrayds"


!az storage account create \
    -n {account_name} \
    -g {resource_group} \
    --query 'provisioningState'

In [ ]:
key = !az storage account keys list \
    --account-name {storage_account_name} \
    -g {resource_group} \
    --query '[0].value'

account_key = str(key[0][1:-1])

!az storage container create \
    --account-name {account_name} \
    --account-key {account_key} \
    --name {container_name}

In [ ]:
# this cell will save the credentials above to notshared/credentials.json and will be used in the following notebooks.


json.dumps({'account_name': account_name,'account_key':account_key,'container_name':container_name,'file_share_name':file_share_name},'../notshared/credentials.json')